In [4]:
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf


In [ ]:
print("TensorFlow version: ", tf.version)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices("GPU")))

## 1. Top 10 strongest 

In [ ]:
data = pd.read_csv('../data/type_top_10_attack.csv')

df = data.copy()

df = pd.DataFrame(df)

df.head()

In [ ]:
df['primary_type'].value_counts()

In [ ]:
df[['primary_type']].head()

## Funktion för battles

In [9]:
def get_first_attacker(p1, p2):
    if p1['speed'] > p2['speed']:
        return p1, p2
    elif p2['speed'] > p1['speed']:
        return p2, p1
    else:
        attacker = np.random.choice([p1, p2])
        defender = p2 if attacker == p1 else p1
        return attacker, defender

# En attack räknas ut såhär:
# (attack / defense) * effectiveness * random_factor
def calculate_attack_damage(attacker, defender, effectiveness):
    random_factor = np.random.uniform(0.85, 1.0)
    attack = attacker['attack']
    defense = defender['defense']
    
    damage = (attack / defense) * effectiveness * random_factor
    return damage

# Räknar ut skadan
def perform_attack(attacker, defender):
    attacker_type = attacker['primary_type']
    effectiveness = defender.get(f'{attacker_type.lower()}_attack_effectiveness', 1)
    
    damage = calculate_attack_damage(attacker, defender, effectiveness)
    defender['hp'] -= damage
    return defender['hp'] <= 0, damage 

def handle_round(attacker, defender):
  if perform_attack(attacker, defender):
      return attacker, defender
  if perform_attack(defender, attacker):
      return defender, attacker
  return None, None

def simulate_battle(p1, p2):
    p1['hp'] = p1['original_hp']
    p2['hp'] = p2['original_hp']
    
    attacker, defender = get_first_attacker(p1, p2)
    while True:
        winner, loser = handle_round(attacker, defender)
        
        if winner:
            return winner, loser


## Plockar ut en sample av pokemon pairs

In [ ]:
n_per_type = 7

sample_pokemon = df.groupby('primary_type').apply(lambda x: x.sample(n=n_per_type, random_state=42)).reset_index(drop=True)

pokemon_pairs = list(itertools.combinations(sample_pokemon.to_dict(orient='records'), 2))
print(f'Number of Battles: {len(pokemon_pairs)}')

In [11]:
try:
    battle_results = []
    battle_id = 1 

    for p1, p2 in pokemon_pairs:
        winner, loser = simulate_battle(p1, p2)
        
        winner_effektiveness = winner.get(f'{loser["primary_type"].lower()}_attack_effectiveness', 1)
        loser_effektiveness = loser.get(f'{winner["primary_type"].lower()}_attack_effectiveness', 1)

        battle_results.append ({
            'battle_id': battle_id,
            'name': winner['name'],
            'pokemon_id': winner['pokedex_number'],
            'type': winner['primary_type'],
            'primary_type': winner['primary_type'],
            'pokemon_class': winner['pokemon_class'],
            'attack': winner['attack'],
            'defense': winner['defense'],
            'speed': winner['speed'],
            'hp': winner['original_hp'],
            'effectiveness': winner_effektiveness,
            'opponent_id': loser['pokedex_number'],
            'opponent_name': loser['name'],
            'opponent_primary_type': loser['primary_type'],
            'attack_first': winner['speed'] > loser['speed'], 
            'is_winner': True,
        })
        battle_results.append({
            'battle_id': battle_id,
            'name': loser['name'],
            'pokemon_id': loser['pokedex_number'],
            'type': loser['primary_type'],
            'primary_type': loser['primary_type'],
            'pokemon_class': loser['pokemon_class'],
            'attack': loser['attack'],
            'defense': loser['defense'],
            'speed': loser['speed'],
            'hp': loser['original_hp'],
            'effectiveness': loser_effektiveness,
            'opponent_id': winner['pokedex_number'],
            'opponent_name': winner['name'],
            'opponent_primary_type': winner['primary_type'],
            'attack_first': loser['speed'] > winner['speed'],
            'is_winner': False
        })
        battle_id += 1

except Exception as e:
    print(f"An error occurred during battle {battle_id} between {p1['name']} and {p2['name']}: {e}")


In [12]:
df_battles = pd.DataFrame(battle_results)
df_battles.dtypes

df_battles.to_csv('../data/battle_results_top_10.csv', index=False)

In [ ]:
df_battles.shape